In [1]:
import pandas as pd
import torch
import numpy as np
from argparse import Namespace
import os
import json

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from torch.utils.data import Dataset
from sklearn.preprocessing import OneHotEncoder

# Multiclass Classification

## Data Loading

In [110]:
task2 = pd.read_csv(f'./data/task2.csv')

In [ ]:
class JobDescriptionDataset(Dataset):
    def __init__(self, df, train, val, test, vectorizer):
        """
        Args:
        review_df (pandas.DataFrame): the dataset
        vectorizer (Vectorizer): vectorizer instantiated from dataset
        """
        self.df = df
        self.train = train
        self.val = val 
        self.test = test 
        self._vectorizer = vectorizer
        self.train_size = len(self.train)
        self.validation_size = len(self.val)
        self.test_size = len(self.test)
        self._lookup_dict = {'train': (self.train, self.train_size),
                            'val': (self.val, self.validation_size),
                            'test': (self.test, self.test_size)}
        self.set_split('train')
        
    @classmethod
    def load_dataset_and_make_vectorizer(cls, name):
        """Load dataset and make a new vectorizer from scratch
        Args:
             name (str): name of task to load
        Returns:
            an instance of ReviewDataset
        """
        df, train, test, val = load_data(name)
        return cls(df, train, test, val, OneHotVectoriser.from_dataframe(df)) 
    
    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer
    
    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe
        Args:
        split (str): one of "train", "val", or "test"
        """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]
        
    def __len__(self):
        return self._target_size
    
    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        Args:
        index (int): the index to the data point
        Returns:
        a dict of the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]
        input_vector = \
        self._vectorizer.vectorize(row.tfidf10)
        class_index = \
        self._vectorizer.class_vocab.lookup_token(row.is_fulltime)
        return {'x_data': input_vector,
                'y_target': class_index}
    
    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        Args:
        batch_size (int)
        Returns:
        number of batches in the dataset
        """
        return len(self) // batch_size

In [ ]:
class Vocabulary(object):
    """Class to process text and extract Vocabulary for mapping"""
    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
        """
        Args:
            token_to_idx (dict): a pre-existingmap of tokens to indices
            add_unk (bool): a flag that indicates whether to add the UNK token
            unk_token (str): the UNK token to add into the Vocabulary
        """
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx
        self._idx_to_token = {idx: token 
                                for token, idx in self._token_to_idx.items()}
        self._add_unk = add_unk
        self._unk_token = unk_token
        self.unk_index = 1
        if add_unk:
            self.unk_index = self.add_token(unk_token)
            
    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx,
                'add_unk': self._add_unk,
                'unk_token': self._unk_token}
    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)
    
    def add_token(self, token):
        """Update mapping dicts based on the token.
        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    
    def lookup_token(self, token):
        """Retrieve the index associated with the token
        or the UNK index if token isn't present.
        Args:
            token (str): the token to look up
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary)
            for the UNK functionality
        """
        if self._add_unk:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]
        
    def lookup_index(self, index):
        """Return the token associated with the index
        Args:
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
        KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]
    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)
    def __len__(self):
        return len(self._token_to_idx)
    
    

In [6]:
class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):

        super(SequenceVocabulary, self).__init__(token_to_idx)

        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)

    def to_serializable(self):
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token': self._unk_token,
                         'mask_token': self._mask_token,
                         'begin_seq_token': self._begin_seq_token,
                         'end_seq_token': self._end_seq_token})
        return contents

    def lookup_token(self, token):
        """Retrieve the index associated with the token 
          or the UNK index if token isn't present.
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary) 
              for the UNK functionality 
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

In [ ]:
class JobDescriptionVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""   
    def __init__(self, word_vocab, job_vocab):
        """
        Args:
            word_vocab (Vocabulary): maps words to integers
            job_vocab (Vocabulary): maps job types to integers
        """
        self.word_vocab = word_vocab
        self.job_vocab = job_vocab

    def vectorize(self, job_description, vector_length=-1):
        """
        Args:
            job_description (str): A job description
            vector_length (int): an argument for forcing the length of index vector
        """
        assert isintance(job_description, str)
        
        indices = [self.self.word_vocab.begin_seq_index]
        indices.extend(self.char_vocab.lookup_token(token) 
                       for token in surname)
        indices.append(self.char_vocab.end_seq_index)

        if vector_length < 0:
            vector_length = len(indices)

        out_vector = np.zeros(vector_length, dtype=np.int64)         
        out_vector[:len(indices)] = indices
        out_vector[len(indices):] = self.char_vocab.mask_index
        
        return out_vector, len(indices)

    @classmethod
    def from_dataframe(cls, surname_df):
        """Instantiate the vectorizer from the dataset dataframe
        
        Args:
            surname_df (pandas.DataFrame): the surnames dataset
        Returns:
            an instance of the SurnameVectorizer
        """
        char_vocab = SequenceVocabulary()
        nationality_vocab = Vocabulary()

        for index, row in surname_df.iterrows():
            for char in row.surname:
                char_vocab.add_token(char)
            nationality_vocab.add_token(row.nationality)

        return cls(char_vocab, nationality_vocab)

    @classmethod
    def from_serializable(cls, contents):
        char_vocab = SequenceVocabulary.from_serializable(contents['char_vocab'])
        nat_vocab =  Vocabulary.from_serializable(contents['nationality_vocab'])

        return cls(char_vocab=char_vocab, nationality_vocab=nat_vocab)

    def to_serializable(self):
        return {'char_vocab': self.char_vocab.to_serializable(), 
                'nationality_vocab': self.nationality_vocab.to_serializable()}

In [25]:
jdv = JobDescriptionVectorizer()

TypeError: __init__() missing 2 required positional arguments: 'word_vocab' and 'job_vocab'

In [103]:
train, test, val = load_data('task_2')
#x_train_tensor = torch.as_tensor(x_data).float()
#y_train_tensor = torch.as_tensor(y_truth).float()

In [104]:
train_x = train.job_description
train_y = train.category.astype('category')

In [105]:
train_y.unique()

['Farming, Animals & Conservation', 'Community Services & Development', 'Hospitality & Tourism', 'Information & Communication Technology', 'Trades & Services', ..., 'Marketing & Communications', 'CEO & General Management', 'Design & Architecture', 'Human Resources & Recruitment', 'Retail & Consumer Products']
Length: 30
Categories (30, object): ['Accounting', 'Administration & Office Support', 'Advertising, Arts & Media', 'Banking & Financial Services', ..., 'Science & Technology', 'Self Employment', 'Sport & Recreation', 'Trades & Services']

# Models

## Vanilla RNN

NameError: name 'args' is not defined

### Interactive

In [77]:
interact(rnn_model_one_hot)

NameError: name 'interact' is not defined

In [78]:
interact(rnn_model_our_embeddings)

NameError: name 'interact' is not defined

In [79]:
interact(rnn_model_pretrained)

NameError: name 'interact' is not defined

## LSTM